## 대법원 판례 크롤링 ##

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
##대법원 특허 판례 크롤링

import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from datetime import datetime
from elasticsearch.exceptions import TransportError, AuthenticationException
from elasticsearch import Elasticsearch, helpers
import re
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

url = "http://www.law.go.kr/DRF/lawSearch.do?OC=tjddms303&target=prec&type=XML&search=2&query=%ED%8A%B9%ED%97%88&"

response = requests.get(url)

# XML 파싱
xml_data = ET.fromstring(response.content)

totalCnt = int(xml_data.find('totalCnt').text)
print(totalCnt)

page = 1
rows = []

page_count = (totalCnt // 20) + (1 if totalCnt % 20 != 0 else 0)
for i in trange(1, page_count + 1):
    try:
        prec_info = xml_data[5:]
    except:
        break
        
    for info in prec_info:
        judicPrecNum = info.find('판례일련번호').text
        case = info.find('사건명').text
        caseNum = info.find('사건번호').text
        sentence_date = info.find('선고일자').text
        court = info.find('법원명').text
        caseInfo = info.find('사건종류명').text
        caseCode = info.find('사건종류코드').text
        judgment = info.find('판결유형').text
        sentence = info.find('선고').text
        judicPrecLink = info.find('판례상세링크').text

        rows.append({'판례일련번호': judicPrecNum,
                    '사건명': case,
                    '사건번호': caseNum,
                    '선고일자': sentence_date,
                    '법원명': court,
                    '사건종류명': caseInfo,
                    '사건종류코드': caseCode,
                    '판결유형': judgment,
                    '선고': sentence,
                    '판례상세링크': judicPrecLink})
    page += 1
    response = urlopen(url + '&page=' + str(page)).read()
    xml_data = ET.fromstring(response)



2276


100%|██████████| 114/114 [00:10<00:00, 11.29it/s]


In [4]:
url = "http://www.law.go.kr/DRF/lawService.do?OC=tjddms303&target=prec&ID=241243&type=HTML&mobileYn="
test = requests.get(url)

soup = BeautifulSoup(test.content, 'html.parser')
print(soup)


<html>
<head>
<script src="/DRF/js/jquery.js" type="text/javascript"></script>
<style>
	body {
	    overflow: hidden; 
	    width: 100%;
		height : 100%;
	}
	
	#bodyDiv {
		width: 100%;
		height: 100%;
		overflow-x : hidden;
		overflow-y : auto;
	}
	
	</style>
</head>
<script type="text/javascript">
	$(document).ready(function(){
		// 모바일일 경우 수행		
		if($("#mobileYn").val() == 'Y'){
			var url = "mobilePrecInfoR.do";
			var param = "precSeq="+$("#precSeq").val(); 
			
			$.ajax({
				url: url,
				type:'POST',
				dataType:'text',
				data: param,
				timeout : 150000,
		        error : function(request,status,error){
		        	//alert("다시 실행해주시기 바랍니다. :: " + status);
		        	location.href = $("#url").val();
		        },
				success: function(responseData, result){
					if(responseData != ''){
						$("#bodyDiv").html(responseData);
					}else{
						location.href = $("#url").val();
					}
					
				}
			});	
		}
	});
</script>
<body bottommargin="0" leftmargin="0" marginheight="0"

In [3]:
rows

[{'판례일련번호': '241243',
  '사건명': '거절결정(특)[이 사건 청구항이 존속기간이 연장될 수 있는 발명에 해당하는지 여부가 문제된 사건]',
  '사건번호': '2021후11070',
  '선고일자': '2024.07.25',
  '법원명': '대법원',
  '사건종류명': '특허',
  '사건종류코드': '400106',
  '판결유형': '판결',
  '선고': '선고',
  '판례상세링크': '/DRF/lawService.do?OC=tjddms303&target=prec&ID=241243&type=HTML&mobileYn='},
 {'판례일련번호': '241347',
  '사건명': '취소결정(특)',
  '사건번호': '2021후10022',
  '선고일자': '2024.05.30',
  '법원명': '대법원',
  '사건종류명': '특허',
  '사건종류코드': '400106',
  '판결유형': '판결',
  '선고': '선고',
  '판례상세링크': '/DRF/lawService.do?OC=tjddms303&target=prec&ID=241347&type=HTML&mobileYn='},
 {'판례일련번호': '241359',
  '사건명': '보관금반환',
  '사건번호': '2023다309549',
  '선고일자': '2024.03.28',
  '법원명': '대법원',
  '사건종류명': '민사',
  '사건종류코드': '400101',
  '판결유형': '판결',
  '선고': '선고',
  '판례상세링크': '/DRF/lawService.do?OC=tjddms303&target=prec&ID=241359&type=HTML&mobileYn='},
 {'판례일련번호': '239215',
  '사건명': '등록무효(특)',
  '사건번호': '2020후10292',
  '선고일자': '2024.01.11',
  '법원명': '대법원',
  '사건종류명': '특허',
  '사건종류코드': '400106',
  '판결유형': '판결'

In [ ]:
## 판례목록을 다운받아서 API에 포함되지 않은 목록 찾기
import re

# 텍스트 파일을 읽는 함수
def read_patent_cases(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # 판례 정보를 담을 리스트
    cases = []

    lines = text.split('\n')
    for line in lines:
        # 앞의 숫자와 점을 제외한 나머지 텍스트만 처리
        line = re.sub(r'^\d+\.', '', line).strip()
        case_data = {}

        # 1. 사건명: 괄호가 있으면 괄호까지, 없으면 숫자 전까지
        if '(' in line:  # 괄호가 있으면 괄호까지 사건명
            case_data['사건명'] = line.split('(')[0].strip() + '(' + line.split('(')[1].split(')')[0] + ')'
        else:  # 괄호가 없으면 숫자가 나오기 전까지 사건명
            case_data['사건명'] = ''.join(re.findall(r'^[^\d]+', line)).strip()

        # 2. 사건번호: 숫자+한글+숫자의 형식
        match_case_num = re.search(r'(\d+[가-힣]+\d+)', line)
        if match_case_num:
            case_data['사건번호'] = match_case_num.group(0)

        # 3. 법원명: 법원이라는 단어가 포함된 부분
        match_court = re.search(r'([가-힣]+법원)', line)
        if match_court:
            case_data['법원명'] = match_court.group(0)

        # 4. 판결일자: 법원명 뒤에 나오는 날짜 (YYYY.MM.DD 형식)
        match_date = re.search(r'(\d{4}\.\s*\d{1,2}\.\s*\d{1,2})', line)
        if match_date:
            case_data['판결일자'] = match_date.group(0).replace(" ", "")

        # 판례 데이터가 모두 채워졌다면 리스트에 추가
        if case_data:
            cases.append(case_data)

    return cases


# 파일 경로 지정
file_path = '판례목록.txt'

# 판례 정보 추출
patent_cases = read_patent_cases(file_path)
patent_cases = patent_cases[:-1]

In [71]:
patent_case_nums = {case['사건번호'] for case in patent_cases}  # patent_cases에서 사건번호 추출
row_case_nums = {row['사건번호'] for row in rows}  # rows에서 사건번호 추출

# 2. patent_cases에는 있지만 rows에는 없는 사건번호를 추출
missing_cases = [case for case in patent_cases if case['사건번호'] not in row_case_nums]

# 3. 결과 출력
for case in missing_cases:
    print(case)

{'사건명': '특허권 압류명령', '사건번호': '2011마2412', '법원명': '대법원', '판결일자': '2012.4.16'}
{'사건명': '특허권처분금지가처분이의', '사건번호': '94다14865', '법원명': '대법원', '판결일자': '1995.2.10'}
{'사건명': '특허모조품제조판매금지가처분이의', '사건번호': '91다24113', '법원명': '대법원', '판결일자': '1992.3.31'}
{'사건명': '특허료납부기간해태면제불허가처분취소', '사건번호': '82누264', '법원명': '대법원', '판결일자': '1982.12.14'}


## 청구항-종속항 ##

In [23]:
import re

file_path = 'C:/Users/Kim Seongeun/Downloads/특허 지식베이스/특허분야_18년/장치-독립항-청구항구조화정보/장치-독립항-공개공보-청구항구조화정보.txt'

open_lists = []


with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        input_text = line.strip()
        data_parts = input_text.split('¶')

        patent_number = data_parts[0]
        content = data_parts[-2]

        content = re.sub(r'<ANY>(.*?)</ANY>', r'\1', content)
        content = re.sub(r'<RELATION_.*?>(.*?)</RELATION_.*?>', r'\1', content)
        content = re.sub(r'(<COMPONENT>)(.*?)(</COMPONENT>)', r'\2', content)
        output_text = re.sub(r'</?[^>]+>', ' ', content)
        output_text = re.sub(r'\s+', ' ', output_text).strip()

        open_lists.append({patent_number:output_text})

In [24]:
open_lists

[{'1019400024294': '내부에 히터가 설치된 상,하부틀과'},
 {'1019400024294': '상,하부틀의 위치하는 단열재와, 상,하부틀과 단열재사이에 분할되게 삽입되어 단열재와 상,하부틀사이에 공간부를 형성하는 중간체와, 상기 단열제의 외측에 부착되는 판체로 구성된 적충체와'},
 {'1019400024294': '상기 적충체를 결착하기 위한 채결 수단과'},
 {'1019400024294': '그 체결수단을 수용하는 적충체의 구멍과 채결수단사이에 열변형을 방지하기 위한 틈새가 형성된 것을 특징으로 하는 쉐도우 마스크 금형'},
 {'1019760000726': '신호를 전송 또는 기록되기 전에 그 신호의 다이너믹 레인지를 압축하고 또 전송후 도는 재생후에 신호의 다이너믹 레인지를 신장하게 되었는 잡음 저감 방식에서 압축기는 압축기의 입력신호 다이너믹 레인지에 직선적으로 관계되어 있는 다이너믹 레인지를 가지는 제1신호성분을 출력으로 발생하는 제1회로부와 제2신호 성분이고, 압축기에 의인정되는 레벨증대가 저입력 신호 레벨에서만 일어나고 그리고 고 입력레벨에서는 이 제2신호성분이 전기한 제1신호성분의 레벨의 약 0.1배 이하의 레벨로되는 것 같은 신형 리미터에 의해 제한되어 있는 제2신호 성분을 출력으로서 발생하는 제2회로부를 구비하고 압축은 압축기의 제1회로부에서의 제1신호성분을 압축기의 제2회로부에서의 제2신호 성분에 의해 증대하는 것에 의해 되고 신장되는 신장기의 입력신호의 다이너믹 레인지에 직선적으로 관계하고 있는 다이너믹 레인지를 가지는 제3신호 성분을 출력으로서 발생하는 제1회로부와 제4신호성분이고, 신장기에서의 인정된 레벨 축소가 신장기에 저입력 신호 레벨시에서만 일어나고, 신장기에의 입력신호의 레벨이 높을 때에만 이 제4신호 성분이 전기한 제3신호 성분의 레벨의 약 0.1배 이하의 레벨로 되게 선형 리미터에 의해 제한되어 있는 제4신호성분을 출력으로 해서 발생하게 되어 있고 또한 전기한 압축기의 제2회로부와 동일 또는 실질상 같은 

In [25]:
open_result = {'content': {}}

for d in open_lists:
    for patent_number, description in d.items():
        if patent_number in open_result['content']:
            open_result['content'][patent_number] += ',' + description
        else:
           open_result['content'][patent_number] = description

In [26]:
open_result

{'content': {'1019400024294': '내부에 히터가 설치된 상,하부틀과,상,하부틀의 위치하는 단열재와, 상,하부틀과 단열재사이에 분할되게 삽입되어 단열재와 상,하부틀사이에 공간부를 형성하는 중간체와, 상기 단열제의 외측에 부착되는 판체로 구성된 적충체와,상기 적충체를 결착하기 위한 채결 수단과,그 체결수단을 수용하는 적충체의 구멍과 채결수단사이에 열변형을 방지하기 위한 틈새가 형성된 것을 특징으로 하는 쉐도우 마스크 금형',
  '1019760000726': '신호를 전송 또는 기록되기 전에 그 신호의 다이너믹 레인지를 압축하고 또 전송후 도는 재생후에 신호의 다이너믹 레인지를 신장하게 되었는 잡음 저감 방식에서 압축기는 압축기의 입력신호 다이너믹 레인지에 직선적으로 관계되어 있는 다이너믹 레인지를 가지는 제1신호성분을 출력으로 발생하는 제1회로부와 제2신호 성분이고, 압축기에 의인정되는 레벨증대가 저입력 신호 레벨에서만 일어나고 그리고 고 입력레벨에서는 이 제2신호성분이 전기한 제1신호성분의 레벨의 약 0.1배 이하의 레벨로되는 것 같은 신형 리미터에 의해 제한되어 있는 제2신호 성분을 출력으로서 발생하는 제2회로부를 구비하고 압축은 압축기의 제1회로부에서의 제1신호성분을 압축기의 제2회로부에서의 제2신호 성분에 의해 증대하는 것에 의해 되고 신장되는 신장기의 입력신호의 다이너믹 레인지에 직선적으로 관계하고 있는 다이너믹 레인지를 가지는 제3신호 성분을 출력으로서 발생하는 제1회로부와 제4신호성분이고, 신장기에서의 인정된 레벨 축소가 신장기에 저입력 신호 레벨시에서만 일어나고, 신장기에의 입력신호의 레벨이 높을 때에만 이 제4신호 성분이 전기한 제3신호 성분의 레벨의 약 0.1배 이하의 레벨로 되게 선형 리미터에 의해 제한되어 있는 제4신호성분을 출력으로 해서 발생하게 되어 있고 또한 전기한 압축기의 제2회로부와 동일 또는 실질상 같은 구성을 가진 제2회로부를 구비하고 신장은 신장기의 제1회로부에서의 제3신호 성분을 신장기의 제2회로부에서의

In [27]:
import json
json_file_path = '청구항_독립항_공개공보.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(open_result, json_file, ensure_ascii=False, indent=4)

In [ ]:
import re

file_path = 'C:/Users/Kim Seongeun/Downloads/특허 지식베이스/특허분야_18년/장치-종속항-청구항구조화정보/장치-종속항-등록공보-청구항구조화정보.txt'

regis_lists = []


with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        input_text = line.strip()
        data_parts = input_text.split('¶')

        patent_number = data_parts[0]
        content = data_parts[-1]

        output_text = re.sub(r'<DIRECTION>(.*?)</DIRECTION>', r'\1', content)
        output_text = re.sub(r'<PATENT_ITEM>(.*?)</PATENT_ITEM>', r'\1', output_text)
        output_text = re.sub(r'</?[^>]+>', ' ', output_text)
        output_text = re.sub(r'\s+', ' ', output_text).strip()

        regis_lists.append({patent_number:output_text})


In [10]:
regis_result = {'content': {}}

for d in regis_lists:
    for patent_number, description in d.items():
        if patent_number in regis_result['content']:
            regis_result['content'][patent_number] += ", " + description
        else:
            regis_result['content'][patent_number] = description

In [ ]:
import json
json_file_path = '청구항_종속항_등록공보.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(regis_result, json_file, ensure_ascii=False, indent=4)

In [ ]:
import re

file_path = 'C:/Users/Kim Seongeun/Downloads/특허 지식베이스/특허분야_18년/장치-종속항-청구항구조화정보/장치-종속항-공개공보-청구항구조화정보.txt'

open_lists = []


with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        input_text = line.strip()
        data_parts = input_text.split('¶')

        patent_number = data_parts[0]
        content = data_parts[-1]

        output_text = re.sub(r'<DIRECTION>(.*?)</DIRECTION>', r'\1', content)
        output_text = re.sub(r'<PATENT_ITEM>(.*?)</PATENT_ITEM>', r'\1', output_text)
        output_text = re.sub(r'</?[^>]+>', ' ', output_text)
        output_text = re.sub(r'\s+', ' ', output_text).strip()

        open_lists.append({patent_number:output_text})



In [ ]:
open_result = {'content': {}}

for d in open_lists:
    for patent_number, description in d.items():
        # 만약 result['content']에 이미 해당 patent_number가 있으면 쉼표로 합침
        if patent_number in open_result['content']:
            open_result['content'][patent_number] += ", " + description
        else:
            open_result['content'][patent_number] = description

In [ ]:
import json
json_file_path = '청구항_종속항_공개공보.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(open_result, json_file, ensure_ascii=False, indent=4)